In [4]:
import azureml.core
from azureml.core import Experiment, Environment, Workspace, Dataset, Datastore, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
import os
import shutil

# check core SDK version number

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.44.0


In [6]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
# get metadata about the workspace
print(ws.name, ws.location, ws.resource_group, sep='\t')
# list the registered datastores
ws.datastores

landcover-ai	eastus	cic_ai


{'solardatablob': {
   "name": "solardatablob",
   "container_name": "solar",
   "account_name": "aiprojects",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'animalopsblobstore2': {
   "name": "animalopsblobstore2",
   "container_name": "animal-ops",
   "account_name": "aiprojects",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'animalopsblobstore': {
   "name": "animalopsblobstore",
   "container_name": "animal-ops",
   "account_name": "aiprojects",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspacefilestore': {
   "name": "workspacefilestore",
   "container_name": "azureml-filestore-0b767baf-fb3d-4e08-a2d6-663739db0e23",
   "account_name": "aiprojects",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspaceworkingdirectory': {
   "name": "workspaceworkingdirectory",
   "container_name": "code-391ff5ac-6576-460f-ba4d-7e03433c68b6",
   "account_name": "aiprojects",
   "protocol": "https",
   "endpoint": "core.w

In [5]:
datastore = Datastore.get(workspace = ws, datastore_name = 'solardatablob')
datastore.unregister()

In [8]:
# register our data share containing image data in this workspace
Datastore.register_azure_blob_container(
    account_key = 'sZ/bw2Viouyp/C0Duhboamqx5VDXNtAm2fyYzrucLsUNk5nQXkvURAMnBeehMiL1xE+LEMTRBeaq+AStNkzzkQ==',
    workspace = ws,
    datastore_name = 'solarDataBlob',
    container_name = 'solar',
    account_name = 'aiprojects')

Datastore name solarDataBlob contains capital letters. They will be converted to lowercase letters.


{
  "name": "solardatablob",
  "container_name": "solar",
  "account_name": "aiprojects",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [ ]:
# create a file dataset that can be used in training
datastore = Datastore.get(workspace = ws, datastore_name = 'solardatablob')
datastore_paths = [(datastore, 'CPK_solar'), (datastore, 'NC_solar')]
cpk_dataset = Dataset.File.from_files(path = datastore_paths[0])
nc_dataset = Dataset.File.from_files(path = datastore_paths[1])

cpk_dataset = cpk_dataset.register(
    workspace=ws,
    name='gee-cpk-solar-data',
    description='training and eval TFRecords for solar arrays exported from GEE',
    create_new_version=True)

nc_dataset = nc_dataset.register(
    workspace=ws,
    name='gee-nc-solar-data',
    description='training and eval TFRecords for solar arrays exported from GEE',
    create_new_version=True)

## Environments

In [7]:
envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)

Name AzureML-responsibleai-0.20-ubuntu20.04-py38-cpu
Name AzureML-responsibleai-0.21-ubuntu20.04-py38-cpu
Name AzureML-PTA-pytorch-1.11-py38-cuda11.3-gpu
Name AzureML-PTA-pytorch-1.11-py38-cuda11.5-gpu
Name AzureML-sklearn-1.0-ubuntu20.04-py38-cpu
Name AzureML-tensorflow-2.6-ubuntu20.04-py38-cuda11-gpu
Name AzureML-tensorflow-2.5-ubuntu20.04-py38-cuda11-gpu
Name AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu
Name AzureML-ACPT-pytorch-1.11-py38-cuda11.3-gpu
Name AzureML-ACPT-pytorch-1.11-py38-cuda11.5-gpu
Name AzureML-pytorch-1.10-ubuntu18.04-py38-cuda11-gpu
Name AzureML-ACPT-pytorch-1.12-py39-cuda11.6-gpu
Name AzureML-ACPT-pytorch-1.12-py38-cuda11.6-gpu
Name AzureML-minimal-ubuntu18.04-py37-cuda11.0.3-gpu-inference
Name AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu
Name AzureML-sklearn-0.24-ubuntu18.04-py37-cpu
Name AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu
Name AzureML-pytorch-1.7-ubuntu18.04-py37-cuda11-gpu
Name AzureML-pytorch-1.8-ubuntu18.04-py37-cuda11-gpu
Name AzureML-py

In [ ]:
base_env = envs.get('AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu')
base_env

In [27]:
!pwd

/mnt/batch/tasks/shared/LS_root/mounts/clusters/mevans1/code/Users/mevans/Solar_UNet


In [34]:
env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.0.3-cudnn8-ubuntu18.04:20221010.v1",
    conda_file="/mnt/batch/tasks/shared/LS_root/mounts/clusters/mevans1/code/Users/mevans/Solar_UNet/envs/conda_env.yml",
    name="solar-training",
    description="Environment created from a Docker image plus Conda environment.",
)



In [35]:
env_docker_conda

{
    "assetId": null,
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220708.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "solar-training",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
      

In [21]:
base_env.name = 'solar-training'
base_env.register(ws)

Environment version is set. Attempting to register desired version. To auto-version, reset version to None.


{
    "assetId": "azureml://locations/eastus/workspaces/0b767baf-fb3d-4e08-a2d6-663739db0e23/environments/solar-training/versions/50",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": "FROM mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.0.3-cudnn8-ubuntu18.04:20221010.v1\n\nENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/tensorflow-2.4\n# Create conda environment\nRUN conda create -p $AZUREML_CONDA_ENVIRONMENT_PATH \\\n    python=3.7 pip=20.2.4\n\n# Prepend path to AzureML conda environment\nENV PATH $AZUREML_CONDA_ENVIRONMENT_PATH/bin:$PATH\n\n# Install pip dependencies\nRUN HOROVOD_WITH_TENSORFLOW=1 \\\n    pip install 'matplotlib>=3.3,<3.4' \\\n                'psutil>=5.8,<5.9' \\\n                'tqdm>=4.59,<4.60' \\\n                'pandas>=1.1,<1.2' \\\n                'scipy>=1.5,<1.6' \\\n

In [ ]:
# create an environment for the first time

envs = Environment.list(workspace = ws)
# well start with a pre-built tensorflow environment
env = envs.get('AzureML-TensorFlow-2.3-GPU')
env

# define packages to be installed using CondaDependencies
# get the packages that are already part of the pre-built environment
conda_dep = env.python.conda_dependencies
# list packages to install
pip_packages = ['matplotlib', 'rasterio', 'tensorboard']

# add each package to the existing conda dependencies
for package in pip_packages:
    conda_dep.add_pip_package(package)

# double check all the packages are there
conda_dep.serialize_to_string()
# conda_dep = CondaDependencies.create(
#     pip_packages=pip_packages)

# Now update the conda dependencies of the python environment
env.python.conda_dependencies=conda_dep

# # Register environment to re-use later
env.name = 'solar-training'
env.register(workspace = ws)

In [25]:
envs = Environment.list(workspace = ws)
# well start with a pre-built tensorflow environment
env = envs.get('AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu')
# env
solar_env = env.clone('test')
conda_dep = CondaDependencies()
# list packages to install
pip_packages = ['matplotlib', 'tensorboard']

# add each package to the existing conda dependencies
for package in pip_packages:
    conda_dep.add_pip_package(package)

conda_dep.add_conda_package('rasterio')

solar_env.python.conda_dependencies=conda_dep

# # Register environment to re-use later
solar_env.name = 'test'
solar_env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": "FROM mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.0.3-cudnn8-ubuntu18.04:20220113.v1\n\nENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/tensorflow-2.4\n\n# Create conda environment\nRUN conda create -p $AZUREML_CONDA_ENVIRONMENT_PATH \\\n    python=3.7 pip=20.2.4\n\n# Prepend path to AzureML conda environment\nENV PATH $AZUREML_CONDA_ENVIRONMENT_PATH/bin:$PATH\n\n# Install pip dependencies\nRUN HOROVOD_WITH_TENSORFLOW=1 \\\n    pip install 'matplotlib>=3.3,<3.4' \\\n                'psutil>=5.8,<5.9' \\\n                'tqdm>=4.59,<4.60' \\\n                'pandas>=1.1,<1.2' \\\n                'scipy>=1.5,<1.6' \\\n                'numpy>=1.10,<1.20' \\\n                'ipykernel~=6.0' \\\n                'azureml-core==1.37.0.post1' \\\n     

In [18]:
# create an environment for the first time

envs = Environment.list(workspace = ws)
# well start with a pre-built tensorflow environment
env = envs.get('AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu')
# env
solar_env = env.clone('test')
# define packages to be installed using CondaDependencies
# get the packages that are already part of the pre-built environment
conda_dep = CondaDependencies()
# list packages to install
pip_packages = ['matplotlib', 'tensorboard']

# add each package to the existing conda dependencies
for package in pip_packages:
    conda_dep.add_pip_package(package)

conda_dep.add_conda_package('rasterio')

# double check all the packages are there
conda_dep.serialize_to_string()
# conda_dep = CondaDependencies.create(
#     pip_packages=pip_packages)

# Now update the conda dependencies of the python environment
solar_env.python.conda_dependencies=conda_dep

# # Register environment to re-use later
solar_env.name = 'solar-training'
solar_env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": "FROM mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.0.3-cudnn8-ubuntu18.04:20220113.v1\n\nENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/tensorflow-2.4\n\n# Create conda environment\nRUN conda create -p $AZUREML_CONDA_ENVIRONMENT_PATH \\\n    python=3.7 pip=20.2.4\n\n# Prepend path to AzureML conda environment\nENV PATH $AZUREML_CONDA_ENVIRONMENT_PATH/bin:$PATH\n\n# Install pip dependencies\nRUN HOROVOD_WITH_TENSORFLOW=1 \\\n    pip install 'matplotlib>=3.3,<3.4' \\\n                'psutil>=5.8,<5.9' \\\n                'tqdm>=4.59,<4.60' \\\n                'pandas>=1.1,<1.2' \\\n                'scipy>=1.5,<1.6' \\\n                'numpy>=1.10,<1.20' \\\n                'ipykernel~=6.0' \\\n                'azureml-core==1.37.0.post1' \\\n     

In [13]:
iterator = iter(env.python.conda_dependencies.conda_packages)

In [20]:
solar_env.name

'solar-training'